<a href="https://colab.research.google.com/github/Nirjhar100/Major-FakeNews/blob/master/cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras import layers
from keras.layers import Dropout
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [0]:
df = pd.read_csv('./clean_data.csv')
print(df.shape)
df.tail()

(31658, 3)


,Unnamed: 0,subreddit,title
31653,19991,0,after confessing to playing hours of breath o...
31654,19992,0,slipknot singer says imagine dragons have repl...
31655,19993,0,couple lies stranger than fiction
31656,19995,0,ambulance theft suspect i should ve jacked a car
31657,19998,0,byu pole vaulter pierces scrotum in horrific a...


In [0]:
X=df['title']
y=df['subreddit']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10,
                                                     random_state=42)

In [0]:
tokenizer = Tokenizer(num_words = 5000)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

vocab_size = len(tokenizer.word_index)+1
print(vocab_size)
print(X_train[1])

25649
[4428, 4429, 4430, 4096, 10, 511]


In [0]:
maxlen = 50
X_train = pad_sequences(X_train, padding = 'post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding = 'post', maxlen = maxlen)
print(X_train[0, :])

[  38 4771    5   29   78   33 4772    2 1493  149    2 1278   12  714
 2774 2659    6  633 2244   10  613    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0]


In [0]:
embedding_dim = 100
#input_dim = X_train.shape[1] #number of features
model = Sequential()

model.add(layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim,
                           input_length = maxlen))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPool1D())
model.add(layers.Dense(32, activation = 'relu'))
model.add(layers.Dense(10, activation = 'relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_26"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_25 (Embedding)     (None, 50, 100)           2564900   
_________________________________________________________________
conv1d_23 (Conv1D)           (None, 46, 128)           64128     
_________________________________________________________________
global_max_pooling1d_22 (Glo (None, 128)               0         
_________________________________________________________________
dense_39 (Dense)             (None, 32)                4128      
_________________________________________________________________
dense_40 (Dense)             (None, 10)                330       
_________________________________________________________________
dense_41 (Dense)             (None, 1)                 11        
Total params: 2,633,497
Trainable params: 2,633,497
Non-trainable params: 0
___________________________________________

In [0]:
history=model.fit(X_train,y_train,epochs = 6, verbose = 1, validation_data=(X_test,y_test),batch_size=1024)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 28492 samples, validate on 3166 samples
Epoch 1/6
28492/28492 [==============================] - 13s 449us/step - loss: 0.6531 - accuracy: 0.6552 - val_loss: 0.5469 - val_accuracy: 0.7979
Epoch 2/6
28492/28492 [==============================] - 13s 441us/step - loss: 0.3850 - accuracy: 0.8449 - val_loss: 0.3384 - val_accuracy: 0.8544
Epoch 3/6
28492/28492 [==============================] - 13s 439us/step - loss: 0.2490 - accuracy: 0.8962 - val_loss: 0.3167 - val_accuracy: 0.8632
Epoch 4/6
28492/28492 [==============================] - 13s 441us/step - loss: 0.1924 - accuracy: 0.9249 - val_loss: 0.3265 - val_accuracy: 0.8617
Epoch 5/6
28492/28492 [==============================] - 13s 440us/step - loss: 0.1484 - accuracy: 0.9456 - val_loss: 0.3566 - val_accuracy: 0.8645
Epoch 6/6
28492/28492 [==============================] - 13s 441us/step - loss: 0.1111 - accuracy: 0.9604 - val_loss: 0.3959 - val_accuracy: 0.8664


In [0]:
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Training Accuracy: 0.9768
Testing Accuracy:  0.8664
